# DoveTrek Route Solver

Finds the optimal hiking route using **Bitmask Dynamic Programming**.

- **Mode A**: Maximize checkpoints visited at a given walking speed
- **Mode B**: Find the minimum walking speed needed to visit all 17 checkpoints

### Rules
- Start at "Start" (10:00), finish at "Finish" by 17:00
- 17 intermediate checkpoints, each with 30-min open/close windows
- 7 minutes dwell time at each checkpoint
- Travel time = `(Distance / Speed) * 60 + (Height_Gain / 10)` minutes
- Waiting at a closed checkpoint for the next open slot is allowed

In [ ]:
import csv
import math
import os
from dataclasses import dataclass

import pandas as pd


@dataclass
class Config:
    """Solver configuration."""
    speed: float = 5.3              # km/h
    dwell: int = 7                  # minutes at each checkpoint
    naismith: float = 10.0          # extra minutes per 100m height gain
    start_time: int = 600           # 10:00 in minutes from midnight
    end_time: int = 1020            # 17:00 in minutes from midnight
    openings_path: str = "CheckpointData/Openings_2025.csv"
    distances_path: str = "DataFrames/Distances_DF_2025_Google Maps_2025-03-02.csv"


cfg = Config()
print(f"Speed: {cfg.speed} km/h | Dwell: {cfg.dwell} min | Naismith: {cfg.naismith} min/100m")
print(f"Window: {cfg.start_time // 60}:{cfg.start_time % 60:02d} - {cfg.end_time // 60}:{cfg.end_time % 60:02d}")

In [ ]:
# ── Data Loading ─────────────────────────────────────────────────────

def load_openings(filepath: str) -> tuple[list[str], list[int], dict[str, list[int]]]:
    """Parse the openings CSV.

    Returns
    -------
    cp_names : list[str]
        Checkpoint names in file order.
    slot_starts : list[int]
        Slot start times in minutes from midnight (e.g. [600, 630, ...]).
    openings : dict[str, list[int]]
        Mapping cp_name -> list of 0/1 per slot.
    """
    with open(filepath, encoding="utf-8-sig") as f:
        reader = csv.reader(f)
        header = next(reader)
        # header: CP, BNG, 1000, 1030, ..., 1700
        slot_labels = header[2:]  # ["1000", "1030", ...]
        slot_starts = [int(s[:2]) * 60 + int(s[2:]) for s in slot_labels]

        cp_names = []
        openings = {}
        for row in reader:
            if not row or not row[0].strip():
                continue
            name = row[0].strip()
            cp_names.append(name)
            openings[name] = [int(x) for x in row[2:]]

    return cp_names, slot_starts, openings


def load_distances(filepath: str) -> dict[tuple[str, str], tuple[float, float]]:
    """Parse distances CSV into {(StartCP, FinishCP): (Distance, Height_Gain)}."""
    dist = {}
    with open(filepath, encoding="utf-8-sig") as f:
        reader = csv.DictReader(f)
        for row in reader:
            key = (row["StartCP"].strip(), row["FinishCP"].strip())
            dist[key] = (float(row["Distance"]), float(row["Height_Gain"]))
    return dist


cp_names, slot_starts, openings = load_openings(cfg.openings_path)
distances = load_distances(cfg.distances_path)

# Display openings
slot_labels = [f"{s // 60}:{s % 60:02d}" for s in slot_starts]
open_df = pd.DataFrame(
    {name: openings[name] for name in cp_names},
    index=slot_labels,
).T
open_df.index.name = "Checkpoint"
display(open_df)

# Display distance sample
dist_df = pd.DataFrame(
    [(s, f, d, h) for (s, f), (d, h) in distances.items()],
    columns=["StartCP", "FinishCP", "Distance", "Height_Gain"],
)
print(f"\n{len(dist_df)} distance records loaded.")
display(dist_df.head(10))

In [ ]:
# ── Precomputation ───────────────────────────────────────────────────

# Separate Start/Finish from the 17 intermediate checkpoints.
# Bitmask indices 0..16 correspond to the 17 intermediates.
intermediate_cps = [n for n in cp_names if n not in ("Start", "Finish")]
N = len(intermediate_cps)
assert N == 17, f"Expected 17 intermediates, got {N}"

# Index mappings
cp_to_idx = {name: i for i, name in enumerate(intermediate_cps)}
idx_to_cp = {i: name for i, name in enumerate(intermediate_cps)}

# All nodes: 0..N-1 = intermediates, N = Start, N+1 = Finish
START = N      # index 17
FINISH = N + 1 # index 18
ALL_NODES = N + 2


def node_name(idx: int) -> str:
    if idx == START:
        return "Start"
    if idx == FINISH:
        return "Finish"
    return idx_to_cp[idx]


def travel_time(from_idx: int, to_idx: int, speed: float) -> float:
    """Travel time in minutes between two nodes at the given speed."""
    key = (node_name(from_idx), node_name(to_idx))
    if key not in distances:
        return float("inf")
    d, h = distances[key]
    return (d / speed) * 60 + (h / cfg.naismith)


def build_travel_matrix(speed: float) -> list[list[float]]:
    """Build ALL_NODES x ALL_NODES travel time matrix."""
    mat = [[float("inf")] * ALL_NODES for _ in range(ALL_NODES)]
    for i in range(ALL_NODES):
        for j in range(ALL_NODES):
            if i != j:
                mat[i][j] = travel_time(i, j, speed)
    return mat


# Build openings boolean array: N x num_slots
num_slots = len(slot_starts)
open_at = [[False] * num_slots for _ in range(N)]
for i in range(N):
    name = intermediate_cps[i]
    for s in range(num_slots):
        open_at[i][s] = openings[name][s] == 1

# Finish openings
finish_open = [openings["Finish"][s] == 1 for s in range(num_slots)]

print(f"Intermediate checkpoints ({N}): {intermediate_cps}")
print(f"Time slots ({num_slots}): {slot_labels}")
print(f"Travel time Start->CP3: {travel_time(START, cp_to_idx['CP3'], cfg.speed):.2f} min")

In [ ]:
# ── Time Window Helpers ──────────────────────────────────────────────

def arrival_to_slot_index(arrival_minutes: float) -> int:
    """Convert arrival time (minutes from midnight) to slot index.

    Matches the existing time_rounder logic: minute-of-hour must be
    *strictly greater than* 30 to advance to the :30 slot.  This means
    an arrival at exactly HH:30 still falls in the HH:00 slot.

    Returns -1 if before the first slot or after the last slot.
    """
    if arrival_minutes < slot_starts[0]:
        return -1
    whole = int(arrival_minutes)
    h, m = divmod(whole, 60)
    slot_time = h * 60 + (30 if m > 30 else 0)
    if slot_time > slot_starts[-1]:
        return num_slots - 1
    for s in range(num_slots):
        if slot_starts[s] == slot_time:
            return s
    return -1


def find_next_open_time(cp_idx: int, arrival_minutes: float) -> float | None:
    """Find the earliest time >= arrival_minutes when checkpoint cp_idx is open.

    Returns the arrival time itself if the current slot is open, or the
    start of the next open slot, or None if no future slot is open.
    """
    slot = arrival_to_slot_index(arrival_minutes)
    if slot < 0:
        slot = 0
    for s in range(slot, num_slots):
        if open_at[cp_idx][s]:
            return max(arrival_minutes, float(slot_starts[s]))
    return None


def can_reach_finish(current_time: float, current_idx: int, tt_matrix: list[list[float]]) -> bool:
    """Check if we can reach Finish from current_idx within an open Finish window."""
    t_to_finish = tt_matrix[current_idx][FINISH]
    finish_arrival = current_time + t_to_finish
    if finish_arrival > cfg.end_time:
        return False
    slot = arrival_to_slot_index(finish_arrival)
    if slot < 0 or slot >= num_slots:
        return False
    for s in range(slot, num_slots):
        if finish_open[s]:
            wait_until = max(finish_arrival, float(slot_starts[s]))
            if wait_until <= cfg.end_time:
                return True
    return False


# Quick sanity check
print("Slot index for 10:00 (600 min):", arrival_to_slot_index(600))    # 0
print("Slot index for 10:25 (625 min):", arrival_to_slot_index(625))    # 0
print("Slot index for 10:30 (630 min):", arrival_to_slot_index(630))    # 0 (matches time_rounder: 30 is NOT > 30)
print("Slot index for 10:31 (631 min):", arrival_to_slot_index(631))    # 1
print("Slot index for 10:45 (645 min):", arrival_to_slot_index(645))    # 1

In [ ]:
# ── Core DP Solver ───────────────────────────────────────────────────

def solve(speed: float) -> tuple[int, list[int]] | None:
    """Bitmask DP solver.

    State: (visited_mask, current_position) -> earliest departure time.
    Goal:  visit as many of the 17 intermediates as possible, then reach
           Finish within its open window.

    Returns (best_mask, route) where route is a list of intermediate
    node indices in visit order, or None if no feasible route exists
    (not even reaching Finish directly from Start).
    """
    tt = build_travel_matrix(speed)

    FULL = (1 << N) - 1  # all 17 bits set
    INF = float("inf")

    # dp[mask][pos] = earliest departure time from pos after visiting mask
    # departure = arrival + dwell
    dp: dict[int, list[float]] = {}
    parent: dict[tuple[int, int], tuple[int, int] | None] = {}

    def get_dp(mask: int) -> list[float]:
        if mask not in dp:
            dp[mask] = [INF] * N
        return dp[mask]

    # ── Initialize: Start -> each intermediate CP ────────────────────
    depart_start = float(cfg.start_time)  # depart Start at 10:00
    for j in range(N):
        arr = depart_start + tt[START][j]
        open_time = find_next_open_time(j, arr)
        if open_time is None:
            continue
        depart_j = open_time + cfg.dwell
        if depart_j > cfg.end_time:
            continue
        if not can_reach_finish(depart_j, j, tt):
            continue
        mask = 1 << j
        row = get_dp(mask)
        if depart_j < row[j]:
            row[j] = depart_j
            parent[(mask, j)] = None  # came from Start

    # ── Main DP loop ─────────────────────────────────────────────────
    # Process masks in order of popcount (fewer bits first)
    masks_by_popcount: dict[int, list[int]] = {}
    for mask in dp:
        pc = bin(mask).count("1")
        masks_by_popcount.setdefault(pc, []).append(mask)

    for pc in range(1, N + 1):
        current_masks = masks_by_popcount.get(pc, [])
        for mask in current_masks:
            row = dp[mask]
            for i in range(N):
                if not (mask & (1 << i)):
                    continue
                if row[i] == INF:
                    continue
                depart_i = row[i]
                # Try extending to each unvisited CP
                for j in range(N):
                    if mask & (1 << j):
                        continue
                    arr_j = depart_i + tt[i][j]
                    if arr_j > cfg.end_time:
                        continue
                    open_time = find_next_open_time(j, arr_j)
                    if open_time is None:
                        continue
                    depart_j = open_time + cfg.dwell
                    if depart_j > cfg.end_time:
                        continue
                    if not can_reach_finish(depart_j, j, tt):
                        continue
                    new_mask = mask | (1 << j)
                    new_row = get_dp(new_mask)
                    if depart_j < new_row[j]:
                        new_row[j] = depart_j
                        parent[(new_mask, j)] = (mask, i)
                        # Register in masks_by_popcount for future processing
                        npc = pc + 1
                        masks_by_popcount.setdefault(npc, [])
                        if new_mask not in masks_by_popcount[npc]:
                            masks_by_popcount[npc].append(new_mask)

    # ── Find the best result ─────────────────────────────────────────
    best_count = -1
    best_finish_time = INF
    best_state = None  # (mask, last_cp_idx)

    for mask, row in dp.items():
        count = bin(mask).count("1")
        for i in range(N):
            if row[i] == INF:
                continue
            # Can we reach Finish?
            finish_arr = row[i] + tt[i][FINISH]
            if finish_arr > cfg.end_time:
                # Maybe we can still reach finish if we wait? No - we already departed.
                # Check if Finish opens in a later slot we can wait for
                fslot = arrival_to_slot_index(finish_arr)
                if fslot < 0 or fslot >= num_slots:
                    continue
                # finish_arr > end_time means we're too late
                continue
            # Find the open slot at or after arrival
            fslot = arrival_to_slot_index(finish_arr)
            if fslot < 0:
                continue
            actual_finish = None
            for s in range(fslot, num_slots):
                if finish_open[s]:
                    actual_finish = max(finish_arr, float(slot_starts[s]))
                    break
            if actual_finish is None or actual_finish > cfg.end_time:
                continue

            # Compare: more CPs > fewer CPs; tiebreak earlier finish
            if (count > best_count) or (count == best_count and actual_finish < best_finish_time):
                best_count = count
                best_finish_time = actual_finish
                best_state = (mask, i)

    if best_state is None:
        return None

    # ── Reconstruct route ────────────────────────────────────────────
    route = []
    state = best_state
    while state is not None:
        route.append(state[1])
        state = parent.get(state)
    route.reverse()

    return best_count, route


print("Solver ready.")

In [ ]:
# ── Route Reconstruction & Output ────────────────────────────────────

def build_route_card(route: list[int], speed: float) -> pd.DataFrame:
    """Build a detailed route card as a pandas DataFrame.

    Parameters
    ----------
    route : list[int]
        Intermediate CP indices in visit order (no Start/Finish).
    speed : float
        Walking speed in km/h.

    Returns
    -------
    pd.DataFrame with columns:
        Leg, From, To, Distance, HeightGain, TravelTime, ArrivalTime,
        TimeSlot, Open, Wait, DepartTime, CumulativeTime
    """
    # Full sequence: Start -> route[0] -> route[1] -> ... -> Finish
    full_seq = [START] + route + [FINISH]

    rows = []
    current_time = float(cfg.start_time)  # depart Start at 10:00

    def fmt_time(minutes: float) -> str:
        h = int(minutes) // 60
        m = int(minutes) % 60
        frac = minutes - int(minutes)
        if frac > 0.005:
            return f"{h}:{m:02d}:{int(frac * 60):02d}"
        return f"{h}:{m:02d}"

    for leg_num, (fi, ti) in enumerate(zip(full_seq[:-1], full_seq[1:])):
        from_name = node_name(fi)
        to_name = node_name(ti)

        key = (from_name, to_name)
        d, h = distances.get(key, (0.0, 0.0))
        tt_min = (d / speed) * 60 + (h / cfg.naismith)

        arrival = current_time + tt_min
        slot_idx = arrival_to_slot_index(arrival)
        slot_label = slot_labels[slot_idx] if 0 <= slot_idx < num_slots else "--"

        if ti == FINISH:
            # Finish node
            is_open = finish_open[slot_idx] if 0 <= slot_idx < num_slots else False
            if is_open:
                wait = 0.0
                depart = arrival  # no dwell at Finish
            else:
                # Wait for next open Finish slot
                depart = arrival
                for s in range(max(0, slot_idx), num_slots):
                    if finish_open[s]:
                        depart = max(arrival, float(slot_starts[s]))
                        slot_label = slot_labels[s]
                        is_open = True
                        break
                wait = depart - arrival
        elif ti < N:
            # Intermediate checkpoint
            is_open = open_at[ti][slot_idx] if 0 <= slot_idx < num_slots else False
            if is_open:
                wait = 0.0
                depart = arrival + cfg.dwell
            else:
                open_time = find_next_open_time(ti, arrival)
                if open_time is not None:
                    wait = open_time - arrival
                    depart = open_time + cfg.dwell
                    ws = arrival_to_slot_index(open_time)
                    slot_label = slot_labels[ws] if 0 <= ws < num_slots else slot_label
                    is_open = True
                else:
                    wait = 0.0
                    depart = arrival + cfg.dwell
        else:
            # Start node (shouldn't be a destination, but handle gracefully)
            is_open = True
            wait = 0.0
            depart = arrival + cfg.dwell

        rows.append({
            "Leg": leg_num + 1,
            "From": from_name,
            "To": to_name,
            "Distance (km)": round(d, 3),
            "Height Gain (m)": round(h, 1),
            "Travel (min)": round(tt_min, 2),
            "Arrival": fmt_time(arrival),
            "Time Slot": slot_label,
            "Open": "Yes" if is_open else "No",
            "Wait (min)": round(wait, 1),
            "Depart": fmt_time(depart),
            "Cumulative (min)": round(depart - cfg.start_time, 2),
        })

        current_time = depart

    return pd.DataFrame(rows)


def print_summary(route: list[int], card: pd.DataFrame, speed: float):
    """Print a summary of the solved route."""
    total_dist = card["Distance (km)"].sum()
    total_height = card["Height Gain (m)"].sum()
    finish_time = card.iloc[-1]["Depart"]
    total_wait = card["Wait (min)"].sum()
    n_cps = len(route)

    print(f"\n{'=' * 60}")
    print(f"  Route Summary")
    print(f"{'=' * 60}")
    print(f"  Checkpoints visited : {n_cps} / {N}")
    print(f"  Walking speed       : {speed} km/h")
    print(f"  Total distance      : {total_dist:.2f} km")
    print(f"  Total height gain   : {total_height:.0f} m")
    print(f"  Total wait time     : {total_wait:.1f} min")
    print(f"  Finish time         : {finish_time}")
    print(f"  Route: Start -> {' -> '.join(node_name(i) for i in route)} -> Finish")
    print(f"{'=' * 60}\n")


print("Route card builder ready.")

In [ ]:
# ── Mode A: Maximize checkpoints at set speed ────────────────────────

print(f"Solving Mode A: Maximize checkpoints at {cfg.speed} km/h ...\n")

result = solve(cfg.speed)

if result is None:
    print("No feasible route found (cannot even reach Finish from Start).")
else:
    best_count, route = result
    card = build_route_card(route, cfg.speed)
    display(card)
    print_summary(route, card, cfg.speed)

In [ ]:
# ── Mode B: Minimum speed for all 17 checkpoints ─────────────────────

print("Solving Mode B: Binary search for minimum speed to visit all 17 checkpoints ...\n")

lo, hi = 3.0, 15.0
precision = 0.01
best_speed = None
best_route = None

while hi - lo > precision:
    mid = (lo + hi) / 2
    result = solve(mid)
    if result is not None and result[0] == N:
        best_speed = mid
        best_route = result[1]
        hi = mid
    else:
        lo = mid

if best_speed is not None:
    print(f"Minimum speed for all {N} checkpoints: {best_speed:.2f} km/h\n")
    card_b = build_route_card(best_route, best_speed)
    display(card_b)
    print_summary(best_route, card_b, round(best_speed, 2))
else:
    print(f"Cannot visit all {N} checkpoints even at {hi} km/h.")
    print("Consider raising the upper speed bound or checking the data.")